**CON LIBRERIA CRYPTOGRAPHY**

In [20]:
from cryptography.fernet import Fernet
path = "/Users/antoniobaio/Desktop/Progetti/ProgettoSSB/userpass.json"

In [21]:
key = Fernet.generate_key()
 
# string the key in a file
with open('filekey.key', 'wb') as filekey:
   filekey.write(key)

In [22]:
# opening the key
with open('filekey.key', 'rb') as filekey:
	key = filekey.read()

# using the generated key
fernet = Fernet(key)

# opening the original file to encrypt
with open(path, 'rb') as file:
	original = file.read()
	
# encrypting the file
encrypted = fernet.encrypt(original)

In [24]:
encrypted

b'gAAAAABkSSsD_uk_quNot2IHYVuBqPs8tvhIwTl23jPX73t2BNBgLCb-AIWkL_nMMMuK6qWIdgd3aTGvXae13vxaagU0g4E9rJ2ngf-wUfeZk2VdXzmhnBYH9oSrO2EPtRW0c8ZNl1RBoWKnJXcvfbebMmvN6UDCKQe2lMGqnKg3iwLDDRC_BdhK_g4RlNv5riVrs-qnYiABkSl0_pfdzHRx-8Q9sQvWHUSEeaD9tmd4ETwh97WjQSFcEqXO9R31j4cK-c03WafSfs4Pg4VwcMURxDn1ku9tqNopVKK2PYYTI1Bi-L0OXxQBWlOl9L1W2pC49vGr3jNoSSXf1biceh-pDa1c9VRlPNkV0TNVxWGfECjP02jbPxqgq6xzv7WtP_dC4I2zItRv9n_ebYqubYhCDYasQOilEi1zYzAGfa-4eANl10tKz4_vXa5hS3hvuEzlWsD5ykRV3QrnT99T5N9a00qoCdwwxjgchfq7Xby6zGFWXs0nSP01yq-32NdjAkhgDqUgA6GwtP7LZywIVk9SdRkLWbT7d6E7D0g9HEhfXbAglqA9WVV82G9Mj2WctnpChfJ4yDu0Zp_uD109nm3izRNZQ9N8Huw9s5A_4NOj1YW5B8_sjlhIuLhmwMjrdDCxuR3lUJ18vC-cuE9yr3M98P6RmZBRPzm8RmKat9sSU3u7TNmXVwCLVz_PzNFwOiR9XrF0cv3AKz2FfqyWN2cA-FCPbZREmEX3J8GHJ6cn6VYkby7mIKgrhb-EGTB_iIVaVeI_sjNDnfbJUUEoBfGgQ2j1lTyGsBuSkeOjIOd_MDXvHl5JMgdy1WSaqDjJfHrUbiRERihPoDg9Cz1wbAAXNvSXlztS7sRICXkS24OCUJcXZzi0uBbgD6ny79LFfYvkFN8gH2KoZisOaKh5_hdZZShg6bIDZK00swogU07qcKq7ZPOV5Yl94Pf27NHAYvBfD2_hKbygujv5vRl4BfDP29nCqusu67mq9Q

In [26]:
# opening the file in write mode and
# writing the encrypted data
with open(path, 'wb') as encrypted_file:
	encrypted_file.write(encrypted)

**CON RSA**

In [34]:
import rsa

# generate public and private keys with
# rsa.newkeys method,this method accepts
# key length as its parameter
# key length should be atleast 16
publicKey, privateKey = rsa.newkeys(16)

In [35]:
publicKey, privateKey

(PublicKey(33221, 65537), PrivateKey(33221, 65537, 11093, 239, 139))

In [36]:
type(publicKey)

rsa.key.PublicKey

In [30]:

# this is the string that we will be encrypting
message = "CIAO"

# rsa.encrypt method is used to encrypt
# string with public key string should be
# encode to byte string before encryption
# with encode method
encMessage = rsa.encrypt(message.encode(),
						publicKey)



In [31]:
print("original string: ", message)
print("encrypted string: ", encMessage)

original string:  CIAO
encrypted string:  b'F1k\x91\x12dj\xad\x12TY\xc3\xf3\xe2\xc2\xbf\xe6\x94<\xb3\x80\xbf\xac\x04\xff[\x83\xae\xaeb\x04yhG\n\xd6\xc0\x97\xf7\xa2\x83\xe5R\xd3\x8b{\xbek\xb1^\xc6G*4D\x00\xb5m\xf6\x88\xbc\xa0c\x9f'


In [32]:
# the encrypted message can be decrypted
# with ras.decrypt method and private key
# decrypt method returns encoded byte string,
# use decode method to convert it to string
# public key cannot be used for decryption
decMessage = rsa.decrypt(encMessage, privateKey).decode()

print("decrypted string: ", decMessage)

decrypted string:  CIAO


**ALTRO**

In [37]:
import secrets
from base64 import urlsafe_b64encode as b64e, urlsafe_b64decode as b64d

from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

In [50]:
backend = default_backend()
iterations = 100_000

def _derive_key(password: bytes, salt: bytes, iterations: int = iterations) -> bytes:
    """Derive a secret key from a given password and salt"""
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(), length=32, salt=salt,
        iterations=iterations, backend=backend)
    return b64e(kdf.derive(password))

def password_encrypt(message: bytes, password: str, iterations: int = iterations) -> bytes:
    salt = secrets.token_bytes(16)
    key = _derive_key(password.encode(), salt, iterations)
    return b64e(
        b'%b%b%b' % (
            salt,
            iterations.to_bytes(4, 'big'),
            b64d(Fernet(key).encrypt(message)),
        )
    )

def password_decrypt(token: bytes, password: str) -> bytes:
    decoded = b64d(token)
    salt, iter, token = decoded[:16], decoded[16:20], b64e(decoded[20:])
    iterations = int.from_bytes(iter, 'big')
    key = _derive_key(password.encode(), salt, iterations)
    return Fernet(key).decrypt(token)

In [51]:
messaggio = input("Inserisci il messaggio: ")
password = input("Inserisci la password: ")

In [52]:
token = password_encrypt(messaggio.encode(), password)
token

b'xU-HdfMi7vaqFOtSWayRIQABhqCAAAAAAGRJMjzrYpAfhR9_2C66Oc6VMrZlQ1a8R7UNqTXdael2OAtz_ncTUZstcgl2TZai9uXu1FRKNv-IymLT_RmkgOhO_Ts_'

In [54]:
password = "acde"

In [ ]:
password_decrypt(token, password).decode()